In [1]:
from os import listdir
from os.path import isfile, join
import io
from zipfile import ZipFile
import requests
import pandas as pd
from datetime import datetime
from IPython.display import display, Markdown, Latex

import geopy.distance

import json

In [2]:
# Base URL de la api de Ingenier@
IGA_API_BASE_URL = "http://ing-acc-movil01.personal.corp/ingenier@/symfony/public/index.php/api"
JOB_SRC_DATA_FOLDER = '..\\src_data\\traffica_insights_jobs\\MT_SantaFe 2024-07-31'
MIN_RSRP_DBM = -103
NEAR_SAMPLES_PERCENT = 90.0

---
# Traffica Insights Data

---

In [3]:
def extract_zip(input_zip):
    input_zip=ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}

def extract_job_src_filenames(folder: str):
    raw = []
    post_processed = None
    files = [f for f in listdir(folder) if isfile(join(folder, f))]
    for file in files:
        if not file.endswith('.zip'):
            continue
        if file.startswith('traffica_insights_job'):
            raw.append(file)
        if file.startswith('traffica_insights_post_processing_job'):
            post_processed = file
    return {
        'raw': raw,
        'post_processed': post_processed,
    }

filenames = extract_job_src_filenames(JOB_SRC_DATA_FOLDER)
raw_filenames = filenames['raw']
post_processed_filename = filenames['post_processed']

# Job
df_job_data = None
df_job_data_items = []
if len(raw_filenames) > 0:
    for raw_filename in raw_filenames:
        path = JOB_SRC_DATA_FOLDER + '\\' + raw_filename
        data = extract_zip(path)
        df_job_data_item = pd.read_csv(io.BytesIO(next(iter(data.values()))), sep=';', decimal=',')
        df_job_data_items.append(df_job_data_item)
        display(Markdown(f'### {raw_filename} ({len(df_job_data_item.index)} rows)'))

    display(Markdown('### Datos Crudos'))
    df_job_data = pd.concat(df_job_data_items, axis=0)
    display(df_job_data)


# Post-Processing Job
df_stats_summary = None
df_stats_total = None
df_stats_filtered = None
df_post_proc_job_data = None
if post_processed_filename is not None:
    path = JOB_SRC_DATA_FOLDER + '\\' + post_processed_filename
    data = extract_zip(path)

    for key in data.keys():
        if key.startswith('stats'):
            df_stats_summary = pd.read_excel(io.BytesIO(data[key]), sheet_name='Resumen')
            df_stats_total = pd.read_excel(io.BytesIO(data[key]), sheet_name='Detalle (Total)')
            df_stats_filtered = pd.read_excel(io.BytesIO(data[key]), sheet_name='Detalle (Filtrado)')
        if key.startswith('data'):
            df_post_proc_job_data = pd.read_csv(io.BytesIO(data[key]))

    display(Markdown(f'### {post_processed_filename}'))
    display(Markdown('### Resumen'))
    display(df_stats_summary)
    display(Markdown('### Detalle (Total)'))
    display(df_stats_total)
    display(Markdown('### Detalle (Filtrado)'))
    display(df_stats_filtered)
    display(Markdown('### Datos Postprocesados'))
    display(df_post_proc_job_data)

### traffica_insights_job_248.zip (548725 rows)

### traffica_insights_job_249.zip (442989 rows)

### traffica_insights_job_250.zip (397160 rows)

### traffica_insights_job_251.zip (129372 rows)

### traffica_insights_job_252.zip (39021 rows)

...

,Cód. Sitio,EMG,eNodeB Id,Inicio Consulta,Final Consulta,Registros Totales,Complet. Datos [%],Distrib. de Registros
0,A07-S375,ZSFPAS,73375,2024-07-26 08:00:00,2024-07-27 22:00:00,5886,91.209795,▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░...
1,A07-S377,ZSFNUN,73377,2024-07-26 08:00:00,2024-07-27 22:00:00,211,7.006579,▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
2,A07-S382,ZSTALB,73382,2024-07-26 08:00:00,2024-07-27 22:00:00,0,0.000000,░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
3,A07-S383,ZSTMAR,73383,2024-07-26 08:00:00,2024-07-27 22:00:00,263,6.961257,▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
4,A07-S390,ZSFCGO,73390,2024-07-26 08:00:00,2024-07-27 22:00:00,349,6.994152,▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
...,...,...,...,...,...,...,...,...
102,A07-S095,ZASVIE,70095,2024-07-26 08:00:00,2024-07-27 22:00:00,13908,91.227339,▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░...
103,A07-S096,ZSFGUA,70096,2024-07-26 08:00:00,2024-07-27 22:00:00,27874,90.017544,▒▒▒░░░░░░░▒▓▒░░░░░▒▓▒░░░░░▒▒▒▒░░░░░░░▒▒▒▒▒░░░░...
104,A07-S097,ZSFHIP,70097,2024-07-26 08:00:00,2024-07-27 22:00:00,35991,73.530702,▓▒░░░▒▓▒░░▒▓▒░░░▓▒░░▓▓░░▒▓▒░░░░▒▒▒▒▒░░░░░░░░░░...
105,A07-S098,ZSTOCE,70098,2024-07-26 08:00:00,2024-07-27 22:00:00,11280,91.202485,▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░...


### Detalle (Filtrado)

,Cód. Sitio,EMG,eNodeB Id,Inicio Consulta,Final Consulta,Registros Totales,Complet. Datos [%],Distrib. de Registros
0,A07-S375,ZSFPAS,73375,2024-07-26 08:00:00,2024-07-27 22:00:00,5886,91.209795,▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░...
1,A07-S377,ZSFNUN,73377,2024-07-26 08:00:00,2024-07-27 22:00:00,211,7.006579,▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
2,A07-S382,ZSTALB,73382,2024-07-26 08:00:00,2024-07-27 22:00:00,0,0.000000,░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
3,A07-S383,ZSTMAR,73383,2024-07-26 08:00:00,2024-07-27 22:00:00,263,6.961257,▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
4,A07-S390,ZSFCGO,73390,2024-07-26 08:00:00,2024-07-27 22:00:00,349,6.994152,▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░...
...,...,...,...,...,...,...,...,...
102,A07-S095,ZASVIE,70095,2024-07-26 08:00:00,2024-07-27 22:00:00,13908,91.227339,▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░...
103,A07-S096,ZSFGUA,70096,2024-07-26 08:00:00,2024-07-27 22:00:00,27874,90.017544,▒▒▒░░░░░░░▒▓▒░░░░░▒▓▒░░░░░▒▒▒▒░░░░░░░▒▒▒▒▒░░░░...
104,A07-S097,ZSFHIP,70097,2024-07-26 08:00:00,2024-07-27 22:00:00,35991,73.530702,▓▒░░░▒▓▒░░▒▓▒░░░▓▒░░▓▓░░▒▓▒░░░░▒▒▒▒▒░░░░░░░░░░...
105,A07-S098,ZSTOCE,70098,2024-07-26 08:00:00,2024-07-27 22:00:00,11280,91.202485,▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░░░░░░░░▒▒▒▒▒▒░░░░░...


### Datos Postprocesados

,Latitud,Longitud,Reg. Totales,Celda Dom. -> Nombre,Celda Dom. -> Reg. Totales,Celda Dom. -> RSRP (Avg),Best Serv. -> Nombre,Best Serv. -> Reg. Totales,Best Serv. -> RSRP (Avg),RSRP Measurement Report Source1 (Avg),...,DL Mean WB CQI (Avg),UL Mean PUSCH RSSI (Avg),UL Mean PUCCH RSSI (Avg),DL Bytes Total Non GBR (Sum),DL Bytes Total GBR (Sum),UL Bytes Total Non GBR (Sum),UL Bytes Total GBR (Sum),UL+DL Bytes Total Non GBR (Sum),UL+DL Bytes Total GBR (Sum),UL+DL Bytes Total (Sum)
0,-31.631918,-60.776026,15,ZSFPASM13,3,-104.333333,ZSFPASN13,1,-97.0,-104.466667,...,7.333333,-100.000000,-104.666667,7252,0,3324,0,10576,0,10576
1,-31.648466,-60.789970,12,ZSFTATL13,5,-95.000000,ZSFTATN13,1,-83.0,-95.583333,...,9.090909,-99.090909,-105.272727,1009300,0,156404,0,1165704,0,1165704
2,-31.630299,-60.774124,14,ZSFPASM13,6,-111.000000,ZSFPASL13,1,-103.0,-107.928571,...,8.333333,-110.500000,-111.000000,352,0,168,0,520,0,520
3,-31.630839,-60.775181,8,ZSFPASL13,3,-105.000000,ZSFPASL13,3,-105.0,-108.000000,...,9.000000,-106.800000,-102.500000,67260,0,71704,0,138964,0,138964
4,-31.618968,-60.800533,82,ZSFHSQM13,20,-105.050000,ZSFHSQO13,1,-97.0,-109.707317,...,7.791667,-113.705882,-111.920000,12433396,0,861036,0,13294432,0,13294432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161644,-31.593967,-60.702715,1,ZSFESPL11,1,-99.000000,ZSFESPL11,1,-99.0,-99.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
161645,-31.594327,-60.705673,1,ZSFESPN11,1,-96.000000,ZSFESPN11,1,-96.0,-96.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
161646,-31.595226,-60.705039,1,ZSFESPM11,1,-71.000000,ZSFESPM11,1,-71.0,-71.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0
161647,-31.594327,-60.701659,1,ZSFESPN11,1,-96.000000,ZSFESPN11,1,-96.0,-96.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0,0


In [4]:
df_ti_raw = None
df_ti_best_serv_cell = None
df_ti_dominant_cell = None

# Raw
if df_job_data is not None:
    df_ti_raw = df_job_data[['SectorName', 'Latitude', 'Longitude', 'RSRP Measurement Report Source1']].rename(columns={ 
        'SectorName': 'sector_name', 
        'Latitude': 'lat', 
        'Longitude': 'lng', 
        'RSRP Measurement Report Source1': 'rsrp',
    }).set_index('sector_name')
    df_ti_raw = df_ti_raw.assign(total_samples = lambda r: 1)
    display(Markdown('### Muestras Crudo'))
    display(df_ti_raw)

# Post-processed
if df_post_proc_job_data is not None:
    df_ti_dominant_cell = df_post_proc_job_data[['Celda Dom. -> Nombre', 'Latitud', 'Longitud', 'Celda Dom. -> RSRP (Avg)', 'Celda Dom. -> Reg. Totales']].rename(columns={ 
        'Celda Dom. -> Nombre': 'sector_name', 
        'Latitud': 'lat', 
        'Longitud': 'lng', 
        'Celda Dom. -> RSRP (Avg)': 'rsrp',
        'Celda Dom. -> Reg. Totales': 'total_samples',
    }).set_index('sector_name')
    display(Markdown('### Muestras Dominant Cell'))
    display(df_ti_dominant_cell)

    df_ti_best_serv_cell = df_post_proc_job_data[['Best Serv. -> Nombre', 'Latitud', 'Longitud', 'Best Serv. -> RSRP (Avg)', 'Best Serv. -> Reg. Totales']].rename(columns={ 
        'Best Serv. -> Nombre': 'sector_name', 
        'Latitud': 'lat', 
        'Longitud': 'lng', 
        'Best Serv. -> RSRP (Avg)': 'rsrp',
        'Best Serv. -> Reg. Totales': 'total_samples',
    }).set_index('sector_name')
    display(Markdown('### Muestras Best Server'))
    display(df_ti_best_serv_cell)

### Muestras Crudo

,lat,lng,rsrp,total_samples
sector_name,,,,
ZSJRINL11,-31.597463,-60.568657,-112,1
ZSJRINL11,-31.597463,-60.568657,-112,1
ZSJRINL13,-31.607022,-60.582175,-94,1
ZSJRINL12,-31.609061,-60.579772,-95,1
ZSJRINL13,-31.617826,-60.589063,-104,1
...,...,...,...,...
EPBARRM14,-31.713967,-60.481539,-112,1
EPBARRM16,-31.688669,-60.506859,-87,1
EPBARRM15,-31.717250,-60.534647,-88,1


### Muestras Dominant Cell

,lat,lng,rsrp,total_samples
sector_name,,,,
ZSFPASM13,-31.631918,-60.776026,-104.333333,3
ZSFTATL13,-31.648466,-60.789970,-95.000000,5
ZSFPASM13,-31.630299,-60.774124,-111.000000,6
ZSFPASL13,-31.630839,-60.775181,-105.000000,3
ZSFHSQM13,-31.618968,-60.800533,-105.050000,20
...,...,...,...,...
ZSFESPL11,-31.593967,-60.702715,-99.000000,1
ZSFESPN11,-31.594327,-60.705673,-96.000000,1
ZSFESPM11,-31.595226,-60.705039,-71.000000,1


### Muestras Best Server

,lat,lng,rsrp,total_samples
sector_name,,,,
ZSFPASN13,-31.631918,-60.776026,-97.0,1
ZSFTATN13,-31.648466,-60.789970,-83.0,1
ZSFPASL13,-31.630299,-60.774124,-103.0,1
ZSFPASL13,-31.630839,-60.775181,-105.0,3
ZSFHSQO13,-31.618968,-60.800533,-97.0,1
...,...,...,...,...
ZSFESPL11,-31.593967,-60.702715,-99.0,1
ZSFESPN11,-31.594327,-60.705673,-96.0,1
ZSFESPM11,-31.595226,-60.705039,-71.0,1


---
# Radios Aproximados de Sectores

---

In [5]:
payload = {
    "reportName": None,
    "siteNames": [],
    "filteringParams": {"SiteVisualizationData.SiteStatus": ["OA"]},
    "selectBySiteNames": False,
    "selectByFilteringParams": True
}

payload['reportName'] = "datos_radio_sector"
x = requests.post(IGA_API_BASE_URL + '/public/report/generic-report', json=payload)
with io.BytesIO(x.content) as fh:
    df_radio_sector_raw = pd.read_excel(fh, "Radios aproximados de sectores")

payload['reportName'] = "datos_outdoor"
x = requests.post(IGA_API_BASE_URL + '/public/report/generic-report', json=payload)
with io.BytesIO(x.content) as fh:
    df_outdoor_raw = pd.read_excel(fh, "Datos outdoor")

In [9]:
df_iga_1 = ((
    df_outdoor_raw[['Código', 'Sector', 'Portadora', 'Estado', 'Latitud Antena', 'Longitud Antena', 'Azimuth Efectivo', 'Ancho Horiz.']]
).rename(columns = {
    'Código': 'site_name', 
    'Sector': 'sector_name', 
    'Portadora': 'tech_band',
    'Estado': 'site_status', 
    'Latitud Antena': 'lat_antenna', 
    'Longitud Antena': 'lng_antenna', 
    'Azimuth Efectivo': 'azimuth', 
    'Ancho Horiz.': 'h_beamwidth',
})
.set_index('sector_name'))

df_iga_2 = (
    df_radio_sector_raw[['Sector', 'Radio Aprox. Sector [m]']]
).rename(columns={
    'Sector': 'sector_name',   
    'Radio Aprox. Sector [m]': 'sector_radius_m',   
}).set_index('sector_name')

df_iga = df_iga_1.join(df_iga_2)
display(df_iga)

,site_name,tech_band,site_status,lat_antenna,lng_antenna,azimuth,h_beamwidth,sector_radius_m
sector_name,,,,,,,,
XRCESEL1G,A01-E133,LTE FDD_AWS_2225,Operativo,-33.142181,-64.348739,39,73,618.0
XRCESEL1H,A01-E133,LTE FDD_AWS_2225,Operativo,-33.142181,-64.348739,159,73,573.0
XRCESEL1I,A01-E133,LTE FDD_AWS_2225,Operativo,-33.142181,-64.348739,279,73,713.0
XRCESEM1G,A01-E133,LTE FDD_1900MHz_975,No Operativo,-33.142181,-64.348739,39,73,618.0
XRCESEM1H,A01-E133,LTE FDD_1900MHz_975,No Operativo,-33.142181,-64.348739,159,73,573.0
...,...,...,...,...,...,...,...,...
SBBORIU12,SPU990,WCDMA1900_9788,No Operativo,-34.790932,-58.239595,180,64,4493.0
SBBORIU13,SPU990,WCDMA1900_9788,No Operativo,-34.790932,-58.239595,300,64,2679.0
SBBORIV21,SPU990,WCDMA850_4358,No Operativo,-34.790932,-58.239595,60,64,3991.0


# Add IGA data to Traffica Insights dataframes

In [10]:
def dist_m(lat1: float, lng1: float, lat2:float, lng2: float) -> float | None:
    try:
        return geopy.distance.great_circle((lat1, lng1), (lat2, lng2)).m
    except:
        return None

In [11]:
def get_sample_footprint_radius(df_ti: pd.DataFrame, min_rsrp_dbm:float, near_samples_percent: float) -> pd.DataFrame:
    df2 = df_ti[(df_ti['rsrp'] >= min_rsrp_dbm)]
    df2 = df2.reset_index()[['sector_name', 'total_samples']].groupby('sector_name').sum().reset_index()
    data = {r[0]: {'sector_name': r[0], 'total_samples': r[1]} for r in df2.values.tolist()}
    for sector_name in data.keys():
        df_sector = df_ti[df_ti.index.isin([sector_name])].sort_values(by='dist_m', ascending=True)
        total_samples = data[sector_name]['total_samples']
        accum_samples = 0
        for r in df_sector.to_dict(orient='records'):
            accum_samples += r['total_samples']
            if 100.0 * accum_samples / total_samples >= near_samples_percent:
                data[sector_name]['footprint_radius_m'] = r['dist_m']
                data[sector_name]['footprint_samples'] = accum_samples
                break
    result = pd.DataFrame(data.values())[['sector_name', 'footprint_radius_m', 'total_samples', 'footprint_samples']].set_index('sector_name')
    return result

In [12]:
df_raw = None
df_best_serv_cell = None
df_dominant_cell = None

# Raw
if df_ti_raw is not None:
    df_raw = df_ti_raw.join(df_iga)
    # Add distance to the site in metres for each sample
    df_raw['dist_m'] = df_raw.apply(lambda r: dist_m(r['lat'], r['lng'], r['lat_antenna'], r['lng_antenna']), axis=1)
    # Add sample footprint radius
    df_raw = df_iga.join(get_sample_footprint_radius(df_raw, MIN_RSRP_DBM, NEAR_SAMPLES_PERCENT))
    df_raw['overshooting_ratio'] = df_raw.apply(lambda r: r['footprint_radius_m'] / r['sector_radius_m'], axis=1)
    # Add overshooting_ratio
    df_raw = df_raw.sort_values(by='overshooting_ratio', ascending=False)
    df_raw = df_raw[df_raw['footprint_radius_m'].notnull()]
    display(df_raw)

# Best serv cell
if df_ti_best_serv_cell is not None:
    df_best_serv_cell = df_ti_best_serv_cell.join(df_iga)
    # Add distance to the site in metres for each sample
    df_best_serv_cell['dist_m'] = df_best_serv_cell.apply(lambda r: dist_m(r['lat'], r['lng'], r['lat_antenna'], r['lng_antenna']), axis=1)
    # Add sample footprint radius
    df_best_serv_cell = df_iga.join(get_sample_footprint_radius(df_best_serv_cell, MIN_RSRP_DBM, NEAR_SAMPLES_PERCENT))
    df_best_serv_cell['overshooting_ratio'] = df_best_serv_cell.apply(lambda r: r['footprint_radius_m'] / r['sector_radius_m'], axis=1)
    # Add overshooting_ratio
    df_best_serv_cell = df_best_serv_cell.sort_values(by='overshooting_ratio', ascending=False)
    df_best_serv_cell = df_best_serv_cell[df_best_serv_cell['footprint_radius_m'].notnull()]
    display(df_best_serv_cell)

# Dominant cell
if df_ti_dominant_cell is not None:
    df_dominant_cell = df_ti_dominant_cell.join(df_iga)
    # Add distance to the site in metres for each sample
    df_dominant_cell['dist_m'] = df_dominant_cell.apply(lambda r: dist_m(r['lat'], r['lng'], r['lat_antenna'], r['lng_antenna']), axis=1)
    # Add sample footprint radius
    df_dominant_cell = df_iga.join(get_sample_footprint_radius(df_dominant_cell, MIN_RSRP_DBM, NEAR_SAMPLES_PERCENT))
    df_dominant_cell['overshooting_ratio'] = df_dominant_cell.apply(lambda r: r['footprint_radius_m'] / r['sector_radius_m'], axis=1)
    # Add overshooting_ratio
    df_dominant_cell = df_dominant_cell.sort_values(by='overshooting_ratio', ascending=False)
    df_dominant_cell = df_dominant_cell[df_dominant_cell['footprint_radius_m'].notnull()]
    display(df_dominant_cell)


,site_name,tech_band,site_status,lat_antenna,lng_antenna,azimuth,h_beamwidth,sector_radius_m,footprint_radius_m,total_samples,footprint_samples,overshooting_ratio
sector_name,,,,,,,,,,,,
ZSFCENO12,A07-S043,LTE FDD_850MHz_2450,Operativo,-31.642728,-60.705930,182,68,231.0,1709.052231,249.0,225.0,7.398495
ZSFCENN12,A07-S043,LTE FDD_700MHz_9360,Operativo,-31.642728,-60.705930,182,68,231.0,1679.278930,535.0,482.0,7.269606
ZSFCENL12,A07-S043,LTE FDD_AWS_2225,Operativo,-31.642728,-60.705930,180,61,231.0,1613.368105,417.0,376.0,6.984278
ZSFCENM12,A07-S043,LTE FDD_1900MHz_976,Operativo,-31.642728,-60.705930,180,61,231.0,1366.474467,1292.0,1163.0,5.915474
ZSTMARO13,A07-S383,LTE FDD_850MHz_2450,Operativo,-31.669405,-60.783144,239,68,484.0,2511.849242,147.0,133.0,5.189771
...,...,...,...,...,...,...,...,...,...,...,...,...
ZSFMDSN12,A07-S265,LTE FDD_700MHz_9360,Operativo,-31.605795,-60.673039,92,68,4935.0,299.171735,154.0,139.0,0.060622
ZSFZASF12,A07-S277,LTE FDD_2600MHz_3250,Operativo,-31.655681,-60.709449,120,62,4830.0,286.189111,3085.0,2777.0,0.059252
ZSFMDSO12,A07-S265,LTE FDD_850MHz_2450,Operativo,-31.605795,-60.673039,92,68,4935.0,281.466169,78.0,71.0,0.057035


,site_name,tech_band,site_status,lat_antenna,lng_antenna,azimuth,h_beamwidth,sector_radius_m,footprint_radius_m,total_samples,footprint_samples,overshooting_ratio
sector_name,,,,,,,,,,,,
ZSFCENM12,A07-S043,LTE FDD_1900MHz_976,Operativo,-31.642728,-60.705930,180,61,231.0,6556.365579,221.0,201.0,28.382535
ZSFCENO12,A07-S043,LTE FDD_850MHz_2450,Operativo,-31.642728,-60.705930,182,68,231.0,5667.351426,25.0,23.0,24.533989
ZSFCENL12,A07-S043,LTE FDD_AWS_2225,Operativo,-31.642728,-60.705930,180,61,231.0,4738.556240,122.0,111.0,20.513230
ZSFCGOL11,A07-S390,LTE FDD_AWS_2225,Operativo,-31.658044,-60.710720,0,60,217.0,2095.253905,6.0,6.0,9.655548
ZSFCENN12,A07-S043,LTE FDD_700MHz_9360,Operativo,-31.642728,-60.705930,182,68,231.0,1979.730733,145.0,131.0,8.570263
...,...,...,...,...,...,...,...,...,...,...,...,...
ZSFMDSN12,A07-S265,LTE FDD_700MHz_9360,Operativo,-31.605795,-60.673039,92,68,4935.0,283.746836,95.0,91.0,0.057497
ZSFNUNM13,A07-S377,LTE FDD_1900MHz_976,Operativo,-31.631500,-60.733601,240,61,3139.0,106.578538,2.0,2.0,0.033953
ZSFWMAL12,A07-S122,LTE FDD_AWS_2225,Operativo,-31.642913,-60.656979,205,66,4131.0,137.778076,131.0,118.0,0.033352


,site_name,tech_band,site_status,lat_antenna,lng_antenna,azimuth,h_beamwidth,sector_radius_m,footprint_radius_m,total_samples,footprint_samples,overshooting_ratio
sector_name,,,,,,,,,,,,
ZSFBELL12,A07-S278,LTE FDD_AWS_2225,Operativo,-31.636399,-60.698240,120,60,245.0,11019.257392,21.0,21.0,44.976561
ZSFCENM12,A07-S043,LTE FDD_1900MHz_976,Operativo,-31.642728,-60.705930,180,61,231.0,4913.297129,216.0,195.0,21.269685
ZSFCENL12,A07-S043,LTE FDD_AWS_2225,Operativo,-31.642728,-60.705930,180,61,231.0,4738.556240,92.0,87.0,20.513230
ZSFCGOL11,A07-S390,LTE FDD_AWS_2225,Operativo,-31.658044,-60.710720,0,60,217.0,2095.253905,3.0,3.0,9.655548
ZSFCENN12,A07-S043,LTE FDD_700MHz_9360,Operativo,-31.642728,-60.705930,182,68,231.0,2133.693744,41.0,41.0,9.236769
...,...,...,...,...,...,...,...,...,...,...,...,...
ZSFZASF12,A07-S277,LTE FDD_2600MHz_3250,Operativo,-31.655681,-60.709449,120,62,4830.0,279.384089,2349.0,2115.0,0.057843
ZSFMDSN12,A07-S265,LTE FDD_700MHz_9360,Operativo,-31.605795,-60.673039,92,68,4935.0,283.746836,79.0,79.0,0.057497
ZSFMDSO12,A07-S265,LTE FDD_850MHz_2450,Operativo,-31.605795,-60.673039,92,68,4935.0,260.204416,37.0,40.0,0.052726


In [13]:
# Save to Excel
with pd.ExcelWriter('output\\overshooting_cell_detection.xlsx') as writer:  
    if df_raw is not None:
        df_raw.to_excel(writer, sheet_name='Raw')
    if df_best_serv_cell is not None:
        df_best_serv_cell.to_excel(writer, sheet_name='Best Serv Cell')
    if df_dominant_cell is not None:
        df_dominant_cell.to_excel(writer, sheet_name='Dominant Cell')